# The `JADE` code: Tutorial #2

*Author:* Mara Attia

***

This tutorial focuses on **atmospheric characterization** using the `JADE` code. It assumes you are already acquainted with the basics of the code, namely that you already went through the first tutorial.

Compared to the first tutorial, the present one makes a more advanced use of the `JADE` code. To employ the terminology of the JADE use cases (see [Attia et al. 2025]()), we will go through **RS1**, **RS2**, and **FA4**. Consequently, we will perform **retrieval modeling** (for the first two use cases) as well as **forward modeling** with **multiple simulations**. 

## Use cases RS1, RS2: Retrieval modeling, interior characterization

The `JADE` code can be employed to **constrain the internal structure** of a planet, as well as its **atmospheric contents**, which represents the **RS1** and **RS2** use cases. Both operations are actually done at the same time using the same routine `routines/compo_retrieval.py`.

Before diving into the code details of how to use the latter script, let us first insist on what is in fact constrainable by `JADE`. Again, we refer the reader to [Attia et al. (2021)](https://ui.adsabs.harvard.edu/abs/2021A%26A...647A..40A/abstract) for the extensive details of the implemented physics. The internal structure of a planet, as modeled by `JADE`, is the following:

- An *iron nucleus* ($\alpha$-Fe) at the very core,
- A *silicate mantle* (perovskite) surrounding it,
- A *light atmosphere* (H/He and a trace amount of metals) topping the solid material.

Therefore, starting from observational measurements of the mass and radius of a planet (as well as its orbital parameters), we will **constrain the mass fraction of the different internal structure components** (except for the atmospheric metallicity, fixed by the user), and in doing so **assess degeneracies** between all these parameters.

***

Even if we will not make use of the `routines/main.py` file as would be done for regular simulations, **an input file is still required**. An example one has been provided for this use case in `input/examples/example_rs.txt`. Have a thorough look at it before proceeding. You may notice it is almost the same as the FC example encountered in the first tutorial, with the following main differences:

- `age = 4000.`: we will constrain the interior at the time variable $t_\bullet = 4\,{\rm Gyr}$. The age parameter controls the intrinsic temperature of the planet $T_{\rm int}$ (and its equilibrium temperature $T_{\rm eq}$ in the `stellar_lum = tabular` mode).
- `lazy_init = True`: fast initialization of the `JADE` code. **This setting should always be switched on for this use case, and for this use case only** (unless you really know what you are doing).
- `planet_sma = 0.05`: the semi-major axis of the planet is $a_\bullet = a(t = t_\bullet) = 0.05\,{\rm AU}$.
- `planet_ecc = 0.2`: the eccentricity of the planet is $e_\bullet = e(t = t_\bullet) = 0.2$.

Compared to the first tutorial, the planet is ten times closer-in and eccentric, which typically corresponds to what would happen shortly after the decoupling from a ZLK-inducing perturber. **The planet parameters in the input file correspond here to the system age**, where the retrieval will be applied (currently observed parameters, denoted with a $\bullet$ subscript throughout the present tutorial). In this regard, `Mpl = 0.06` here means that the present mass used for the retrieval is $M_{\rm p,\,\bullet} = 0.06\,M_{\rm J} \simeq 19 M_\oplus$. This is conceptually different from the regular usage of the `JADE` code, in which the orbital parameters and the planet mass in the input file correspond to the starting point of the simulation (at `t_init`, typically after the disk dispersal, like in the first tutorial).

We are conscious that the system age is often not that well constrained in scientific literature. If you have no clue about the age of your star, you can just **set it to a relatively high value (e.g., `age = 6000.`)** as (i) most observed exoplanets have old hosts, (ii) $T_{\rm int}$ drops quite quickly (in $\sim 1\,{\rm Gyr}$) to $\lesssim 100\,{\rm K}$ and plateaus at $\lesssim 50\,{\rm K}$ after $4 - 8\,{\rm Gyr}$ ([Mordasini 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...638A..52M/abstract)), so it has a minimal influence on the internal structure. Nonetheless, **it is important to accurately define `Lbol`** (in `stellar_lum = analytic` mode), since it has a major influence on the retrieval. You can compute it using $L_{\rm bol} = 16 \pi \sigma_{\rm B} \sqrt{1 - e_\bullet^2} a_\bullet^2 T_{\rm eq,\,\bullet}^4$ ([Attia et al. 2021](https://ui.adsabs.harvard.edu/abs/2021A%26A...647A..40A/abstract)), where $\sigma_{\rm B}$ is the Stefan–Boltzmann constant.

*Note:* the values set in the `Mcore`, `YHe`, and `fmantle` parameters of the input file do not matter, as they will be constrained by the retrieval. Nonetheless, it is important to fix a `Zmet` value (trace amount, $Z \leq 10^{-4}$, see discussion in [Attia et al. 2025]()).

***

Let us know have a look at `routines/compo_retrieval.py`, which will perform the internal structure constraining using an MCMC exploration (by means of the `emcee` package, [Foreman-Mackey et al. 2013](https://ui.adsabs.harvard.edu/abs/2013PASP..125..306F/abstract)). All you have to do is to **fill in the various fields of the `USER INPUT` section**. They have already been prefilled for this tutorial. Again, have a thorough look at the file, especially the `USER INPUT` section and the comments there, before proceeding. In particular, you can see via the `Rp_med`, `Rp_sig_up`, and `Rp_sig_dn` fields that the observational constraint on the planet radius is set to $R_{\rm p,\,\bullet} = 3.8^{+0.2}_{-0.1}\,R_\oplus$.

**Warning:** if you run the retrieval locally, be careful with the number of cores `nc` you use for the MCMC. The latter will call the atmospheric structure integrator within the log-probability function, which is itself parallelized on a certain number of cores defined by the `parallel` setting in the input file. Hence, make sure employing `nc` $\times$ `parallel` cores does not overload your computer. In the event you are running the retrieval on a cluster, the number of cores you ask for (using, e.g., `NUM_THREADSPROCESSES` with [Slurm](https://slurm.schedmd.com/documentation.html)) should be `nc` $\times$ `parallel`.

***

Once all the relevant data are entered in the `USER INPUT` section, run the script on the terminal with the input file as an argument (relative to `input/`), after you make sure you are in the `routines/` folder.

    python compo_retrieval.py examples/example_rs.txt

Various information will be printed on the standard output. Depending on the requested number of samples (defined by the `ns` setting) and workers (`nw`), the MCMC can take a long time (few hours, even days, so it is not a bad idea to run it on a cluster). You can also run it in successive chunks through the `reuse` setting. Since the intention of this tutorial is pedagogical, the results are already provided in `saved_data/examples/example_rs/`. In particular, the retrieval values are in `fit_results.log` within the latter folder.

Even though the `compo_retrieval.py` routine conveniently allows to plot the results (using the `chain_path` and `corner_path` settings), we will do it manually here by loading and manipulating the MCMC backend `mcmc.h5` in the aforementioned folder in order to break down the different steps (the routine essentially does the same thing as what we will do here). You could also choose to procede in this way to have more control on the postprocessing and plotting stages.

In [ ]:
# Importing necessary packages

import emcee
import corner
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Loading the MCMC results

#--------------------------------
# Use the same values for these 5 variables as in 'compo_retrieval.py'

mcmc_path = '../saved_data/examples/example_rs/mcmc.h5'
nw        = 16
Rp_med    = 3.8
Rp_sig_up = 0.2
Rp_sig_dn = 0.1

#--------------------------------
# Constructing the sampler

NDIM    = 3 
logP    = lambda x:x #Dummy function just to initialize the sampler
backend = emcee.backends.HDFBackend(mcmc_path)
sampler = emcee.EnsembleSampler(nw, NDIM, logP, backend=backend)
samples = sampler.get_chain()

In [ ]:
# Postprocessing

#--------------------------------
# Burn-in phase (half of the samples)

ns = backend.iteration
nb = ns//2
flat_samples = sampler.get_chain(discard=nb, flat=True)
print(f'Applying burn-in phase (discarding first {nb} iterations)... Shape after burn-in/flattening: \
{flat_samples.shape}.')

#--------------------------------
# Reformatting the samples to have the following shape

#... clean_samples[:, 0]: atmosphere mass fraction
#... clean_samples[:, 1]: silicate mantle mass fraction
#... clean_samples[:, 2]: iron nucleus mass fraction
#... clean_samples[:, 3]: atmospheric helium mass fraction
#... clean_samples[:, 4]: computed planet radius (returned as a blob in the log-probability function of the MCMC)

clean_samples        = np.zeros([flat_samples.shape[0], flat_samples.shape[1] + 2])
clean_samples[:, :2] = flat_samples[:, :2]
clean_samples[:, 2]  = 1. - (flat_samples[:, 0] + flat_samples[:, 1])
clean_samples[:, 3]  = flat_samples[:, 2]
clean_samples[:, 4]  = sampler.get_blobs(discard=nb, flat=True)

#--------------------------------
# Cleaning nonphysical configurations

clean_samples = clean_samples[(clean_samples[:, 2] >= 0.) & (clean_samples[:, 4] > -np.inf)]
print(f'Cleaning nonphysical configurations... Shape after cleaning/reformatting: {clean_samples.shape}.')

In [ ]:
# Plotting the chains

#... Burn-in samples are in red
#... Kept samples are in blue

fig, axes = plt.subplots(NDIM, figsize=(10, 1.5*NDIM), sharex=True, dpi=300)

labels = [r'$f_{\rm H/He}$', r'$f_{\rm Si}$', '$Y$']

for i in range(NDIM):
    ax = axes[i]
    x = np.arange(1, ns + 1)
    y = samples[:, :, i]
    ax.plot(x[:nb + 1], y[:nb + 1], c='tomato',     alpha=0.3)
    ax.plot(x[nb:],     y[nb:],     c='dodgerblue', alpha=0.3)
    ax.set_xlim(1, ns)
    ax.set_ylabel(labels[i])

axes[-1].set_xlabel('Step Number')

fig.tight_layout()
plt.show()

In [ ]:
# Plotting the corner plot

#... Medians and +/- 1 sigma envelopes, as derived from the retrieval, are the black dashed vertical lines
#... Median of the observed radius is the orange vertical line (bottom right)
#... +/- 1 sigma envelope of the observed radius is the two orange dotted vertical lines (bottom right)

fig, axes = plt.subplots(NDIM + 2, NDIM + 2, figsize=(10, 10), dpi=300)
fig = corner.corner(clean_samples, quantiles=(0.159, 0.5, 0.841), fig=fig, plot_datapoints=False, verbose=False)
ax = np.array(axes).reshape((NDIM + 2, NDIM + 2))

ax[-1][-1].vlines(Rp_med, 0, 1, transform=ax[-1][-1].get_xaxis_transform(), colors='orange')
ax[-1][-1].vlines([Rp_med - Rp_sig_dn, Rp_med + Rp_sig_up], 0, 1, transform=ax[-1][-1].get_xaxis_transform(), 
                  colors='orange', linestyles='dotted')

for _ax in ax.flatten():
    _ax.tick_params(rotation=0.)

labels = [r'$f_{\rm H/He}$', r'$f_{\rm Si}$', r'$f_{\rm Fe}$', r'$Y$', r'$R_{\rm p}$ [$R_\oplus$]']
for i in range(NDIM + 1):
    ax[-1, i].set_xlabel(labels[i])
    ax[i + 1, 0].set_ylabel(labels[i + 1])

fig.tight_layout()
plt.show()